Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [228]:
import logging
import functools
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic
from enum import Enum
from dataclasses import dataclass
from tqdm.auto import tqdm
from collections import namedtuple

logging.basicConfig(level=logging.DEBUG)

In [229]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [230]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## First Greedy Algorithm

In [248]:
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))
while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))
logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)"
)
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

INFO:root:result: Found a path of 46 steps, total length 4436.03km


## Problem model

In [232]:
class country(str, Enum):
    it = 'cities/italy.csv'
    cn = 'cities/china.csv'
    ru = 'cities/russia.csv'
    us = 'cities/us.csv'
    vn = 'cities/vanuatu.csv'

class tspProblem:
    def __init__(self, country: str):
        self.CITIES = pd.read_csv(country, header=None, names=['name', 'lat', 'lon'])
        self.DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
        for c1, c2 in combinations(CITIES.itertuples(), 2):
            self.DIST_MATRIX[c1.Index, c2.Index] = self.DIST_MATRIX[c2.Index, c1.Index] = geodesic(
                (c1.lat, c1.lon), (c2.lat, c2.lon)
            ).km
            
    def counter(fn):
        """Simple decorator for counting number of calls"""

        @functools.wraps(fn)
        def helper(*args, **kargs):
            helper.calls += 1
            return fn(*args, **kargs)

        helper.calls = 0
        return helper

    @counter        
    def tsp_cost(self, tsp: np.ndarray):
        assert tsp[0] == tsp[-1]
        assert set(tsp) == set(range(len(self.CITIES)))

        tot_cost = 0
        for c1, c2 in zip(tsp, tsp[1:]):
            tot_cost += self.DIST_MATRIX[c1, c2]
        return tot_cost
        

## My Greedy Algorithm
It's a "DFS" that only consider the N closest city as next possible vertex, not feasible since is a NP-hard problem 

In [ ]:
Tsp = namedtuple('Tsp', ['path','dist', 'partialDist'])

class tspProblemDFS(tspProblem):
    def __init__(self, country, n = 2):
        self.p = super().__init__(country)
        self.N = n
        self.bestTsp = Tsp(None, None, None)
        
    def stepDistance(self, c1: int, c2: int):
        return self.DIST_MATRIX[c1,c2]
    
    def dfs_R(self, tsp = Tsp(np.array([0]), None, 0), visited = np.array([0])):
        if tsp.path.size == len(self.CITIES):
            tspDist = tsp.partialDist + self.stepDistance(tsp.path[-1], 0)
            if self.bestTsp.dist == None or self.bestTsp.dist > tspDist:
                self.bestTsp = Tsp(np.append(tsp.path, [0]), None, tspDist)
            return
        else:
            closestNCity = set(np.setdiff1d(np.argsort(self.DIST_MATRIX[tsp.path[-1],:]), visited, assume_unique=True)[:self.N].flatten())
            while bool(closestNCity):
                nextCity = [closestNCity.pop()]
                newPartialDist = tsp.partialDist + self.stepDistance(tsp.path[-1], nextCity[0])
                if self.bestTsp.dist != None and self.bestTsp.dist < newPartialDist:
                    return
                newTsp = Tsp(np.append(tsp.path, nextCity), None, newPartialDist)
                self.dfs_R(newTsp, np.append(visited, nextCity))
                
problem = tspProblemDFS(country.it)
problem.dfs_R()
ic(problem.bestTsp)
        

## Evolutionary Algorithm

In [ ]:
class tspProblemEA(tspProblem):
    @dataclass
    class Individual:
        def __init__(self, genome, cost):
            self.genome: np.ndarray = genome
            self.fitness: float = cost
    
    def parent_selection(self, population) -> Individual:
        candidates = sorted(np.random.choice(population, 2), key=lambda i: i.fitness)
        return candidates[0]
    
    def mutation(self, p: Individual) -> Individual:
        genome = p.genome.copy()
        idx1 = np.random.randint(1, len(self.CITIES)-3)
        idx2 = np.random.randint(idx1+2, len(self.CITIES)-1)
        genome[idx1:idx2] = genome[idx2-1:idx1-1:-1]
        return self.Individual(genome, self.tsp_cost(genome))

In [246]:
def ea(problem: tspProblemEA, generations = 1000, population_size = 1000, offspring_size=100):
    population = [(lambda x: problem.Individual(x, problem.tsp_cost(x)))((lambda x: np.append( x, x[0]))(np.random.permutation(len(problem.CITIES)))) for _ in range(population_size)]
        
    for g in tqdm(range(generations)):
        offspring = list()
        for _ in range(offspring_size):
            offspring.append(problem.mutation(problem.parent_selection(population)))
        population.extend(offspring)
        population.sort(key=lambda i: i.fitness)
        population = population[:population_size]
        
    return (population[0], problem.tsp_cost.calls)

In [ ]:
countries = [country.vn, country.it, country.us, country.ru, country.cn]
for c in countries:
    problem = tspProblemEA(c)
    winner, cost = ea(problem)
    ic(float(winner.fitness), c)

  0%|          | 0/1000 [00:00<?, ?it/s]